<a href="https://colab.research.google.com/github/arijc76/nn-learn/blob/master/cnn_lenet5_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets
from bokeh.plotting import figure
from bokeh.io import show
from bokeh.models import LinearAxis, Range1d
import numpy as np

In [0]:
# Hyperparameters
num_epochs = 6
num_classes = 10
batch_size = 100
learning_rate = 0.001

In [0]:
# transforms to apply to the data
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

In [0]:
# Data Loader

train_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('/files', train=True, download=True,
                              transform=trans), batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('/files', train=False, download=True,
                              transform=trans), batch_size=batch_size, shuffle=False)

In [0]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(5 * 5 * 16, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out


In [0]:
model = ConvNet()

In [18]:
# Debugging

#from torchvision import models
#model = models.vgg16()
print(model)

ConvNet(
  (layer1): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (drop_out): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [0]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [20]:
# Train the model
total_step = len(train_loader)
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Run the forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())

        # Backprop and perform Adam optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track the accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))

Epoch [1/6], Step [100/600], Loss: 0.2960, Accuracy: 93.00%
Epoch [1/6], Step [200/600], Loss: 0.1230, Accuracy: 95.00%
Epoch [1/6], Step [300/600], Loss: 0.1356, Accuracy: 94.00%
Epoch [1/6], Step [400/600], Loss: 0.1309, Accuracy: 96.00%
Epoch [1/6], Step [500/600], Loss: 0.1446, Accuracy: 94.00%
Epoch [1/6], Step [600/600], Loss: 0.1978, Accuracy: 94.00%
Epoch [2/6], Step [100/600], Loss: 0.3903, Accuracy: 94.00%
Epoch [2/6], Step [200/600], Loss: 0.0571, Accuracy: 99.00%
Epoch [2/6], Step [300/600], Loss: 0.0979, Accuracy: 97.00%
Epoch [2/6], Step [400/600], Loss: 0.1698, Accuracy: 94.00%
Epoch [2/6], Step [500/600], Loss: 0.1991, Accuracy: 95.00%
Epoch [2/6], Step [600/600], Loss: 0.2861, Accuracy: 93.00%
Epoch [3/6], Step [100/600], Loss: 0.1269, Accuracy: 97.00%
Epoch [3/6], Step [200/600], Loss: 0.0349, Accuracy: 100.00%
Epoch [3/6], Step [300/600], Loss: 0.2512, Accuracy: 95.00%
Epoch [3/6], Step [400/600], Loss: 0.1793, Accuracy: 96.00%
Epoch [3/6], Step [500/600], Loss: 0.08

In [21]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format((correct / total) * 100))

Test Accuracy of the model on the 10000 test images: 98.98 %
